# Introduction to Machine Learning
# Exercise Part 3

Written by Morgan Schwartz and David Van Valen.  


---

Some code cells will be marked with 
```
##########################
######## To Do ###########
##########################
```

This indicates that you are being asked to write a piece of code to complete the notebook.

# Part 3: Feature Engineering

Classical machine learning methods often turn to manual feature engineering to extract elements of the data that the model will use for prediction. So far we have relied on the raw data alone, but in some cases well designed features can produce a better model than raw data alone.

In [ ]:
import imageio as iio
import skimage
import sklearn.model_selection
import sklearn.utils
import sklearn.metrics
import sklearn.preprocessing
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import tqdm.auto
import copy

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
        
import tensorflow_addons as tfa

from tensorflow.keras.layers import Input, Flatten, Dense, Activation, BatchNormalization, Conv2D, MaxPool2D, Softmax
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model

## General setup
You need to run all of the code cells from here down to the "Introduction to image filters" section. These blocks of code were directly taken from the prvious notebook so feel free to skip down to the image filter section once you have started them running to load the dataset.

### Load dataset
We will load the same dataset as the previous notebook.

In [ ]:
data_dir = 'data/CellCycle'

In [ ]:
# Load dataframe with sample info
df = pd.read_csv(os.path.join(data_dir, 'img.lst'), sep='\t', header=None)
df = df.rename(columns={1: 'class', 2: 'filepath'})
df['channel'] = df['filepath'].str.split('/',expand=True)[2].str.split('_', expand=True)[1].str.slice(2,3)
df['id'] = df['filepath'].str.split('/',expand=True)[2].str.split('_', expand=True)[0]
df.head()

In [ ]:
# Load data stack
ims = []
ys = []
for i, g in df.groupby('id'):
    im = []
    for _, r in g.iterrows():
        im.append(iio.imread(os.path.join(data_dir, r['filepath'])))
    ims.append(np.stack(im, axis=-1))
    ys.append(r['class'])
    
X_data = np.stack(ims)
y_data = np.stack(ys)
print('X shape:', X_data.shape)
print('y shape:', y_data.shape)

In [ ]:
def extract_classes(X_data, y_data, classes):
    """For a given dataset of categorical labels, this data
    selects the datapoints associated with the classes of interest
    and returns them stacked in a new array
    
    Args:
        X_data (np.array): Array of x data
        y_data (np.array): Array of categorical y data
        classes (list): List of categorical classes to select
        
    Returns:
        np.array: X data after extracting the classes of interest 
        np.array: y data after selecting the classes of interest
    """
    X, y = [], []
    for c in classes:
        # Identify the indicies of the relevant class
        idx = y_data == c
        # Select the X and y data accordingly
        X.append(X_data[idx, ..., 0:1])
        y.append(y_data[idx])

    # Restack the arrays
    X = np.concatenate(X)
    y = np.concatenate(y)
    return X, y

In [ ]:
X, y = extract_classes(X_data, y_data, [4, 5])

print(X.shape, y.shape)

In [ ]:
y[y == 4] = 0
y[y == 5] = 1

In [ ]:
def balance_classes(X, y, minority_id):
    """For a given minority class id, upsample the minority class
    to match the number of samples in the majority class
    
    Args:
        X (np.array): Array of raw data
        y (np.array): Array of class labels
        minority_id (int): Integer of the minority class to be upsampled
        
    Returns:
        np.array: X
        np.array: y
    """
    # Split the X and y arrays into sub arrays containing 
    # 1) only the minority samples and 2) the remaining samples
    x_max = X[y != minority_id]
    y_max = y[y != minority_id]
    x_min = X[y == minority_id]
    y_min = y[y == minority_id]
    
    # Use sklearn.utils.resample to samples from the minority sample array 
    # to match the number of samples in the second array
    x_min, y_min = sklearn.utils.resample(x_min, y_min, n_samples=x_max.shape[0])
    
    # Concatenate the upsampled array 1 with the remaining array 2
    X = np.concatenate([x_max, x_min])
    y = np.concatenate([y_max, y_min])
    
    # Shuffle arrays to randomize sample order
    X, y = sklearn.utils.shuffle(X, y)
    
    return X, y

# Create dataset builder
def build_dataset(X, y, batch_size=1, seed=1, train_size=0.8):
    # Create train/test splits
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
        X, y, 
        train_size=train_size, 
        random_state=seed)
    
    # Balance classes in each split
    X_train, y_train = balance_classes(X_train, y_train, 1)
    X_test, y_test = balance_classes(X_test, y_test, 1)
    
    # Convert y data to categorical
    y_train = tf.keras.utils.to_categorical(y_train)
    y_test = tf.keras.utils.to_categorical(y_test)

    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

    train_dataset = train_dataset.shuffle(256).batch(batch_size)
    test_dataset = test_dataset.batch(batch_size)
    
    return {
        'train': train_dataset,
        'test': test_dataset
    }

### Model configuration

In [ ]:
# Define the linear classifier
def create_linear_classifier():
    """Defines a basic linear classifier to predict 2 classes and 
    compiles the model with the following:
    - Optimizer = Adam
    - Loss = Categorical Crossentropy
    - Metrics = Recall and precision for each class
    """
    inputs = Input((X.shape[1], X.shape[2], 1),
                   name='linear_classifier_input')
    x = Flatten()(inputs)
    x = Dense(2)(x)
    x = Softmax(axis=-1)(x)
    model = Model(inputs=inputs, outputs=x)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-3, clipnorm=0.001),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics = [
                      tf.keras.metrics.Recall(class_id=0),
                      tf.keras.metrics.Recall(class_id=1),
                      tf.keras.metrics.Precision(class_id=0),
                      tf.keras.metrics.Precision(class_id=1)
                  ])
    return model

In [ ]:
# Define training parameters
n_epochs=32

In [ ]:
def train_model(model, data, n_epochs, model_path):
    """Fits a model using data from a `DatasetBuilder`. 
    The weights of the model are saved in a folder according
    to `model_path`
    
    Args:
        model (tensorflow model): Model from `create_linear_classifier`
        data (DatasetBuilder): Dataset builder object 
        n_epochs (int): Number of epochs to train for
        model_path (str): Path to save model weights
    """
    model.fit(
        data['train'],
        epochs=n_epochs,
        verbose=1,
    )
    return model

### Benchmarking functions

In [ ]:
def benchmark_performance(y_true, y_pred):
    """Calculates recall, precision, f1 and a confusion matrix for sample predictions
    
    Args:
        y_true (list): List of integers of true class values
        y_pred (list): List of integers of predicted class value
        cm_norm (optional): 'true' to return a normalized confusion matrix.
            None to return an raw confusion matrix
            
    Returns:
        dict: Dictionary with keys `recall`, `precision`, `f1`, and `cm`
    
    """
    _round = lambda x: round(x, 3)
    
    metrics = {
        'recall': _round(sklearn.metrics.recall_score(y_true, y_pred)),
        'precision': _round(sklearn.metrics.precision_score(y_true, y_pred)),
        'f1': _round(sklearn.metrics.f1_score(y_true, y_pred)),
        'cm': sklearn.metrics.confusion_matrix(y_true, y_pred, normalize=None),
        'cm_norm': sklearn.metrics.confusion_matrix(y_true, y_pred, normalize='true')
    }
    
    return metrics

In [ ]:
def plot_metrics(metrics, name, ax=None):
    """Plots a confusion matrix with summary statistics listed above the plot
    
    The annotations on the confusion matrix are the total counts while
    the colormap represents those counts normalized to the total true items
    in that class.
    
    Args:
        metrics (dict): Dictionary output of `benchmark_performance`
        name (str): Title for the plot
        ax (optional, matplotlib subplot): Subplot axis to plot onto. 
            If not provided, a new plot is created
        classes (optional, list): A list of the classes to label the X and y 
            axes. Defaults to [0, 1] for a two class problem.
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(5,5))
    cb = ax.imshow(metrics['cm_norm'], cmap='Greens', vmin=0, vmax=1)
    
    classes = np.arange(metrics['cm'].shape[0])
    plt.xticks(range(len(classes)), classes)
    plt.yticks(range(len(classes)), classes)
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('True Label')

    for i in range(len(classes)):
        for j in range(len(classes)):
            color='green' if metrics['cm_norm'][i,j] < 0.5 else 'white'
            ax.annotate('{}'.format(metrics['cm'][i,j]), (j, i),
                        color=color, va='center', ha='center')

    _ = plt.colorbar(cb, ax=ax)
    _ = ax.set_title(
            '{}\n'\
            'Recall: {}\n'\
            'Precision: {}\n'\
            'F1 Score: {}\n'\
            ''.format(name, metrics['recall'], metrics['precision'], metrics['f1'])
        )

## Introduction to image filters

Image filters operate by taking a small kernel (or matrix) and applying it to each pixel in the image to compute a new value. As an example, this is the identity kernel 
$$
\begin{bmatrix}
    0 & 0 & 0 \\
    0 & 1 & 0 \\
    0 & 0 & 0 \\
\end{bmatrix}
$$
which when applied to an image will not result in any changes to the data.

Filters can produce a variety of effects on images depending on how the kernel is configured. This can range from blurring an image to extracting edges. Filtered images can contain data that is more informative to the model when distinguishing between classes.

In [ ]:
im = X[np.random.randint(X.shape[0]),...,0]
fig, ax = plt.subplots(1, 3, figsize=(10, 4))

ax[0].imshow(im)
ax[0].set_title('Original')

ax[1].imshow(skimage.filters.gaussian(im))
ax[1].set_title('Gaussian')

ax[2].imshow(skimage.filters.laplace(im))
ax[2].set_title('Laplace')

The effect of a filter can also be influenced by changing the sigma parameter which modifies the size of the kernel, as shown below for the Gaussian filter.

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(15, 4))
sigmas = [1, 2, 5, 10]

for i, s in enumerate(sigmas):
    ax[i].imshow(skimage.filters.gaussian(im, sigma=s))
    ax[i].set_title('Sigma = {}'.format(s))

A variety of filters are made available through the `skimage.filters` [module](https://scikit-image.org/docs/stable/api/skimage.filters.html). In this part of the exercise, we are going to explore how filters can be applied to images in order to extract features for model prediction. While we are going to work with the filters that are easily available through skimage, there are many other transformations that can be applied to images.

In [ ]:
from inspect import getmembers, isfunction

# List all functions in the skimage.filters module to get a list of available filters
[m[0] for m in getmembers(skimage.filters, isfunction)]

<div class="alert alert-block alert-info">

## Task 3.1

Test a variety of the available filters from the `skimage` module. Whenever you are making a modification to an image, you should check the results to make sure that errors are not introduced while generating the transformation. 

Ultimately we are going to use model performance to select the best features for our classification task, but you should be familiar with the output of any filters that you are using. The goal of this next section is to identify a set of candidate filters from which one will be chosen that you think will lead to better classification results on the two classes we are trying to distinguish.

Keep the following things you may want to keep in mind as you approach this problem
- Look at several randomly selected images from the two classes when you are testing a filter
- Explore the effect of parameters available for each filter

**Challenge**: While this task could be approached by testing filters one at a time, consider writing a for loop to rapidly test filters in an automated fashion.


**Tip**: Within a jupyter notebook, you can run `function?` to look at the documentation for that function. Check out the example below.

</div>

In [ ]:
skimage.filters.gaussian?

In [ ]:
##########################
######## To Do ###########
##########################

# Put your code for testing filters here

<div class="alert alert-block alert-success"><h2>Checkpoint 2</h2>
    
Please 👍 the "Checkpoint 2" slack thread when you reach this checkpoint.

We'll discuss as a group which filters seem like they may be most effective at distinguishing our two classes. Please be prepared with your top two or three candidates. 

*Bonus:* Collect screenshots of your filtering results to share with the group. Look out for a slack message for instructions on how to submit images.
    
</div>

<div class="alert alert-block alert-info">

## Task 3.2
Starting with a few of your top choices for filters, train a model on each filter variation and compare the results. Again as an extra challenge, you can write a for loop to automate this process.

You will roughly need to follow these steps:
- Create a new version of `X` and `y` with the filter applied. Make sure that you are working on a new copy of the original data each time you apply a filter. You can use `copy.deepcopy` to create a new copy of an array.
- Create a new `DatasetBuilder` object with the filtered images
    ```
    with tf.device('CPU:0'):
        dataset = build_dataset(X, y, batch_size=64, seed=seed, train_size=train_size)
    ```
- Train a model using the functions `create_linear_classifier` and `train_model` that are defined in an earlier section of the notebook
- Benchmark and collect the metrics for each model variant 
</div>

In [ ]:
# You may want to use these standard parameters from the previous notebook
seed = 10
train_size = 0.8

In [ ]:
##########################
######## To Do ###########
##########################

# Your code for training and comparing models here

<div class="alert alert-block alert-success"><h2>Checkpoint</h2>
    
Please 👍 the "Checkpoint 2" slack thread when you reach this checkpoint.

We'll discuss as a group what filters worked best to distinguish between the two classes. When you're ready, you can submit your best model to this [google form](https://docs.google.com/forms/d/e/1FAIpQLSfVblJCnprXct0IrS87cGw4ijpH2h6bKvcXqj7RNraMrSP3PA/viewform?usp=sf_link). 
    
</div>

## Bonus

If you have extra time, try one of the following challenges:
- How does your selected filter perform on another pair of classes in this dataset?
- Can you automate the search for the top performing filter by writing a function that ultimately reports the top performer?